In [ ]:
# =============================================================================
# =============================================================================
# INTEGRATED BAYESIAN OPTIMIZATION PIPELINE FOR CHEMICAL EXPERIMENTS
# =============================================================================
# =============================================================================
#
# WORKFLOW:
#   Phase 1: Feature Selection (Cells 1-26)
#            → Checkpoint saved
#   Phase 2: Bayesian Optimization (Cells 27+)
#            → Iterative experiment proposal
#
# =============================================================================


# =============================================================================
# CELL 1: IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import pickle
from datetime import datetime
from pathlib import Path

warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel, Matern
from sklearn.pipeline import Pipeline
from scipy.stats import norm
from scipy.optimize import minimize
from itertools import combinations

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Create output directory
OUTPUT_DIR = Path('bo_pipeline_output')
OUTPUT_DIR.mkdir(exist_ok=True)

print("=" * 70)
print("INTEGRATED BAYESIAN OPTIMIZATION PIPELINE")
print("=" * 70)
print(f"Output directory: {OUTPUT_DIR}")
print("✓ Imports complete")


# =============================================================================
# =============================================================================
# PHASE 1: FEATURE SCREENING
# =============================================================================
# =============================================================================

print("\n" + "=" * 70)
print("PHASE 1: FEATURE SCREENING FOR BAYESIAN OPTIMIZATION")
print("=" * 70)


# =============================================================================
# CELL 2: CONFIGURATION - PHASE 1
# =============================================================================

# ┌─────────────────────────────────────────────────────────────────────────┐
# │ MODIFY THIS SECTION FOR YOUR DATA                                       │
# └─────────────────────────────────────────────────────────────────────────┘

# Data file settings
PHASE1_DATA_FILE = 'data.xlsx'  # Your Excel file with initial experiments
PHASE1_SHEET_NAME = 'data'      # Sheet name
HEADER_ROW = 5                   # Row number where headers are (0-indexed)

# Data structure settings
SPLIT_KEYWORD = "PREDICTED OPTIMUM RUNS"  # Keyword to split data (or None)
STOP_FEATURE = "Batch ID"                  # Column name where features end

# Response settings
RESPONSE_COLUMN = "Downy Leak"   # Your response variable name
MAXIMIZE_RESPONSE = False        # True if higher is better, False if lower is better

# Feature selection settings
TARGET_FEATURES = 4              # Target number of features for BO (3-5 recommended)

# Thresholds for feature selection
CORRELATION_STRONG = 0.4
CORRELATION_MODERATE = 0.2
VIP_IMPORTANT = 1.0
VIP_MODERATE = 0.8
MULTICOLLINEARITY_THRESHOLD = 0.7
INTERACTION_THRESHOLD = 0.3

print("=" * 60)
print("PHASE 1 CONFIGURATION")
print("=" * 60)
print(f"  Data file: {PHASE1_DATA_FILE}")
print(f"  Response: {RESPONSE_COLUMN}")
print(f"  Target features: {TARGET_FEATURES}")
print(f"  Optimization: {'Maximize' if MAXIMIZE_RESPONSE else 'Minimize'}")


# =============================================================================
# CELL 3: LOAD AND CLEAN DATA
# =============================================================================

print("=" * 60)
print("LOADING DATA")
print("=" * 60)

# Load Excel file
xls = pd.ExcelFile(PHASE1_DATA_FILE, engine='openpyxl')
df = pd.read_excel(xls, sheet_name=PHASE1_SHEET_NAME, header=HEADER_ROW)

print(f"✓ Loaded sheet: '{PHASE1_SHEET_NAME}'")
print(f"  Initial shape: {df.shape}")


# =============================================================================
# CELL 4: SPLIT DATAFRAME AT KEYWORD (if applicable)
# =============================================================================

if SPLIT_KEYWORD:
    split_index = df.index[df['Run'] == SPLIT_KEYWORD].tolist()
    
    if split_index:
        split_index = split_index[0]
        df_initial = df.iloc[:split_index]
        df_optimum = df.iloc[split_index+1:]
        print(f"✓ Split at '{SPLIT_KEYWORD}'")
        print(f"  df_initial: {len(df_initial)} rows")
        print(f"  df_optimum: {len(df_optimum)} rows")
    else:
        print(f"⚠️ '{SPLIT_KEYWORD}' not found, using all data")
        df_initial = df.copy()
        df_optimum = pd.DataFrame()
else:
    df_initial = df.copy()
    df_optimum = pd.DataFrame()
    print("✓ No split keyword specified, using all data")


# =============================================================================
# CELL 5: CLEAN DATAFRAMES
# =============================================================================

print("=" * 60)
print("CLEANING DATA")
print("=" * 60)

# Clean df_initial
df_initial = df_initial.drop(index=0, errors='ignore')  # Drop units row if present
df_initial = df_initial.dropna(how='all')
df_initial = df_initial.reset_index(drop=True)

# Clean df_optimum if exists
if len(df_optimum) > 0:
    df_optimum = df_optimum.dropna(how='all')
    df_optimum = df_optimum.reset_index(drop=True)

# For Phase 1 feature selection, use only initial experiments
# (df_optimum might have been run with different feature sets)
df_phase1 = df_initial.copy()

print(f"✓ Using {len(df_phase1)} experiments for feature selection")


# =============================================================================
# CELL 6: DEFINE FEATURE COLUMNS
# =============================================================================

print("=" * 60)
print("IDENTIFYING FEATURES")
print("=" * 60)

columns = df_phase1.columns.tolist()

if STOP_FEATURE and STOP_FEATURE in columns:
    feature_list = columns[:columns.index(STOP_FEATURE)]
    print(f"✓ Features up to '{STOP_FEATURE}':")
else:
    feature_list = [c for c in columns if c != RESPONSE_COLUMN]
    print(f"⚠️ '{STOP_FEATURE}' not found, using all columns except response")

# Remove 'Run' or index columns if present
feature_list = [f for f in feature_list if f.lower() not in ['run', 'index', 'unnamed: 0']]

for i, feat in enumerate(feature_list, 1):
    print(f"    {i}. {feat}")


# =============================================================================
# CELL 7: IDENTIFY FEATURE TYPES (Binary vs Continuous)
# =============================================================================

print("=" * 60)
print("FEATURE TYPE CLASSIFICATION")
print("=" * 60)

# Get numeric columns from feature list
numeric_features = df_phase1[feature_list].select_dtypes(include=[np.number]).columns.tolist()
numeric_features = [c for c in numeric_features if c != RESPONSE_COLUMN]

# Classify each feature
binary_cols = []
continuous_cols = []
binary_mappings = {}

print("\nClassification (Binary: 2 unique values, Continuous: 3+ values):")
print("-" * 60)

for col in numeric_features:
    n_unique = df_phase1[col].nunique()
    
    if n_unique == 2:
        binary_cols.append(col)
        unique_vals = sorted(df_phase1[col].dropna().unique())
        mapping = {unique_vals[0]: 0, unique_vals[1]: 1}
        df_phase1[col] = df_phase1[col].map(mapping)
        binary_mappings[col] = mapping
        print(f"  {col}: BINARY")
        print(f"      {unique_vals[0]} → 0, {unique_vals[1]} → 1")
    else:
        continuous_cols.append(col)
        print(f"  {col}: CONTINUOUS ({n_unique} unique values)")

print("\n" + "-" * 60)
print(f"SUMMARY: {len(binary_cols)} binary, {len(continuous_cols)} continuous")

BINARY_FEATURES = binary_cols
CONTINUOUS_FEATURES = continuous_cols


# =============================================================================
# CELL 8: PREPARE X AND y
# =============================================================================

print("=" * 60)
print("DATA PREPARATION")
print("=" * 60)

feature_cols = BINARY_FEATURES + CONTINUOUS_FEATURES

# Create X and y
X = df_phase1[feature_cols].copy()
y = df_phase1[RESPONSE_COLUMN].copy()

# Drop rows with missing response
valid = ~y.isnull()
n_dropped = (~valid).sum()
X = X[valid].reset_index(drop=True)
y = y[valid].reset_index(drop=True)

# Store original (unscaled)
X_original = X.copy()
y_original = y.copy()

print(f"\nDataset:")
print(f"  Samples: {len(X)}")
if n_dropped > 0:
    print(f"  Dropped (missing response): {n_dropped}")

print(f"\nFeatures ({len(feature_cols)} total):")
print(f"  Binary: {len(BINARY_FEATURES)}")
print(f"  Continuous: {len(CONTINUOUS_FEATURES)}")
print(f"\nResponse: {RESPONSE_COLUMN}")
print(f"Samples/Features ratio: {len(X)/len(feature_cols):.1f}")


# =============================================================================
# CELL 9: STANDARDIZE FEATURES (for Phase 1 analysis)
# =============================================================================

print("=" * 60)
print("STANDARDIZATION")
print("=" * 60)

scaler_phase1 = StandardScaler()
X_scaled = X.copy()

if CONTINUOUS_FEATURES:
    X_scaled[CONTINUOUS_FEATURES] = scaler_phase1.fit_transform(X[CONTINUOUS_FEATURES])
    print(f"✓ Standardized {len(CONTINUOUS_FEATURES)} continuous features")

# For modeling: scale binary to [-1, +1]
X_model = X_scaled.copy()
for col in BINARY_FEATURES:
    X_model[col] = X_model[col] * 2 - 1

print(f"✓ Binary features: 0/1 (scaled to [-1,+1] for modeling)")


# =============================================================================
# CELL 10: VISUAL INSPECTION - SCATTER PLOTS
# =============================================================================

print("=" * 60)
print("VISUAL INSPECTION: Feature vs Response")
print("=" * 60)

n_feat = len(feature_cols)
n_cols_plot = min(4, n_feat)
n_rows_plot = int(np.ceil(n_feat / n_cols_plot))

fig, axes = plt.subplots(n_rows_plot, n_cols_plot, figsize=(4*n_cols_plot, 3.5*n_rows_plot))
axes = axes.flatten() if n_feat > 1 else [axes]

for i, col in enumerate(feature_cols):
    ax = axes[i]
    corr = X[col].corr(y)
    
    if col in BINARY_FEATURES:
        for val in [0, 1]:
            data = y[X[col] == val]
            ax.boxplot([data], positions=[val], widths=0.6)
        ax.set_xticks([0, 1])
        ax.set_xlabel(f'{col} (binary)')
    else:
        ax.scatter(X[col], y, alpha=0.6, edgecolors='black', linewidth=0.5)
        z = np.polyfit(X[col], y, 1)
        p = np.poly1d(z)
        x_line = np.linspace(X[col].min(), X[col].max(), 100)
        ax.plot(x_line, p(x_line), 'r--', linewidth=2)
        ax.set_xlabel(col)
    
    ax.set_ylabel(RESPONSE_COLUMN)
    ax.set_title(f'r = {corr:.3f}', fontsize=10)

for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle(f'Features vs {RESPONSE_COLUMN}', fontsize=12, y=1.02)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase1_scatter_plots.png', dpi=150, bbox_inches='tight')
plt.show()


# =============================================================================
# CELL 11: RESPONSE DISTRIBUTION
# =============================================================================

print("=" * 60)
print("RESPONSE DISTRIBUTION")
print("=" * 60)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax1 = axes[0]
ax1.hist(y, bins=15, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(y.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {y.mean():.2f}')
ax1.axvline(y.median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {y.median():.2f}')
ax1.set_xlabel(RESPONSE_COLUMN)
ax1.set_ylabel('Frequency')
ax1.set_title('Response Distribution')
ax1.legend()

ax2 = axes[1]
ax2.boxplot(y, vert=True)
ax2.set_ylabel(RESPONSE_COLUMN)
ax2.set_title('Response Box Plot')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase1_response_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nResponse statistics:")
print(f"  Min:    {y.min():.4f}")
print(f"  Max:    {y.max():.4f}")
print(f"  Mean:   {y.mean():.4f}")
print(f"  Median: {y.median():.4f}")
print(f"  Std:    {y.std():.4f}")


# =============================================================================
# CELL 12: METHOD 1 - CORRELATION ANALYSIS
# =============================================================================

print("=" * 60)
print("METHOD 1: PEARSON CORRELATION")
print("=" * 60)

correlations = X.corrwith(y)

corr_df = pd.DataFrame({
    'feature': feature_cols,
    'correlation': correlations.values,
    'abs_corr': np.abs(correlations.values),
    'type': ['binary' if f in BINARY_FEATURES else 'continuous' for f in feature_cols],
    'direction': ['Positive' if c > 0 else 'Negative' for c in correlations.values]
}).sort_values('abs_corr', ascending=False).reset_index(drop=True)

corr_df['rank_corr'] = range(1, len(corr_df) + 1)

def corr_strength(r):
    if abs(r) >= CORRELATION_STRONG: return 'Strong'
    elif abs(r) >= CORRELATION_MODERATE: return 'Moderate'
    else: return 'Weak'

corr_df['strength'] = corr_df['correlation'].apply(corr_strength)

print("\nResults:")
print(corr_df[['rank_corr', 'feature', 'correlation', 'direction', 'strength', 'type']].to_string(index=False))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
colors = ['forestgreen' if c > 0 else 'crimson' for c in corr_df['correlation']]
ax1.barh(corr_df['feature'][::-1], corr_df['abs_corr'][::-1], color=colors[::-1])
ax1.axvline(x=CORRELATION_STRONG, color='green', linestyle='--', linewidth=2, label=f'Strong ({CORRELATION_STRONG})')
ax1.axvline(x=CORRELATION_MODERATE, color='orange', linestyle='--', linewidth=1.5, label=f'Moderate ({CORRELATION_MODERATE})')
ax1.set_xlabel('|Correlation|')
ax1.set_title('Feature-Response Correlation')
ax1.legend(loc='lower right')

ax2 = axes[1]
ax2.barh(corr_df['feature'][::-1], corr_df['correlation'][::-1], color=colors[::-1])
ax2.axvline(x=0, color='black', linewidth=1)
ax2.set_xlabel('Correlation (with sign)')
ax2.set_title('Direction of Effect')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase1_correlation_analysis.png', dpi=150, bbox_inches='tight')
plt.show()


# =============================================================================
# CELL 13: METHOD 2 - LASSO (Automatic Selection)
# =============================================================================

print("=" * 60)
print("METHOD 2: LASSO REGRESSION")
print("=" * 60)

lasso = LassoCV(cv=5, max_iter=10000, random_state=RANDOM_STATE)
lasso.fit(X_model, y)

lasso_df = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': lasso.coef_,
    'abs_coef': np.abs(lasso.coef_),
    'selected': lasso.coef_ != 0,
    'type': ['binary' if f in BINARY_FEATURES else 'continuous' for f in feature_cols]
}).sort_values('abs_coef', ascending=False).reset_index(drop=True)

lasso_df['rank_lasso'] = range(1, len(lasso_df) + 1)

print(f"\nLasso alpha: {lasso.alpha_:.4f}")
print(f"Features selected: {lasso_df['selected'].sum()}/{len(lasso_df)}")
print("\nResults:")
print(lasso_df[['rank_lasso', 'feature', 'coefficient', 'selected', 'type']].to_string(index=False))

selected_by_lasso = lasso_df[lasso_df['selected']]['feature'].tolist()
print(f"\n✓ Lasso selected: {selected_by_lasso}")

# Plot
plt.figure(figsize=(10, 6))
colors = ['forestgreen' if s else 'lightgray' for s in lasso_df['selected']]
plt.barh(lasso_df['feature'][::-1], lasso_df['abs_coef'][::-1], color=colors[::-1])
plt.xlabel('|Coefficient|')
plt.title('Lasso Coefficients (Green = Selected)')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase1_lasso_analysis.png', dpi=150, bbox_inches='tight')
plt.show()


# =============================================================================
# CELL 14: METHOD 3 - PLS WITH VIP SCORES
# =============================================================================

print("=" * 60)
print("METHOD 3: PLS (VIP Scores)")
print("=" * 60)

max_comp = min(5, len(feature_cols), len(X) - 1)
cv_scores = []

print("Finding optimal components...")
for n in range(1, max_comp + 1):
    scores = cross_val_score(PLSRegression(n_components=n), X_model, y, cv=5, scoring='r2')
    cv_scores.append(scores.mean())
    print(f"  {n} components: CV R² = {scores.mean():.4f}")

optimal_comp = np.argmax(cv_scores) + 1
print(f"\n✓ Optimal: {optimal_comp} components (CV R² = {max(cv_scores):.4f})")

pls = PLSRegression(n_components=optimal_comp)
pls.fit(X_model, y)

def calc_vip(model):
    t, w, q = model.x_scores_, model.x_weights_, model.y_loadings_
    m, p = w.shape
    ss = np.sum(t**2, axis=0) * q.flatten()**2
    total_ss = np.sum(ss)
    vip = np.zeros(m)
    for i in range(m):
        weight = sum((w[i,j]**2) * ss[j] / np.sum(w[:,j]**2) for j in range(p))
        vip[i] = np.sqrt(m * weight / total_ss)
    return vip

vip_scores = calc_vip(pls)

pls_df = pd.DataFrame({
    'feature': feature_cols,
    'VIP': vip_scores,
    'type': ['binary' if f in BINARY_FEATURES else 'continuous' for f in feature_cols]
}).sort_values('VIP', ascending=False).reset_index(drop=True)

pls_df['rank_pls'] = range(1, len(pls_df) + 1)

def vip_category(v):
    if v >= VIP_IMPORTANT: return 'Important'
    elif v >= VIP_MODERATE: return 'Moderate'
    else: return 'Less Important'

pls_df['category'] = pls_df['VIP'].apply(vip_category)

print("\nResults:")
print(pls_df[['rank_pls', 'feature', 'VIP', 'category', 'type']].to_string(index=False))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
colors = ['darkgreen' if v >= VIP_IMPORTANT else 'orange' if v >= VIP_MODERATE else 'lightcoral' 
          for v in pls_df['VIP']]
ax1.barh(pls_df['feature'][::-1], pls_df['VIP'][::-1], color=colors[::-1])
ax1.axvline(x=VIP_IMPORTANT, color='green', linestyle='--', linewidth=2, label=f'Important ({VIP_IMPORTANT})')
ax1.axvline(x=VIP_MODERATE, color='orange', linestyle='--', linewidth=1.5, label=f'Moderate ({VIP_MODERATE})')
ax1.set_xlabel('VIP Score')
ax1.set_title('PLS Variable Importance')
ax1.legend()

ax2 = axes[1]
ax2.plot(range(1, max_comp + 1), cv_scores, 'bo-', linewidth=2, markersize=8)
ax2.axvline(x=optimal_comp, color='red', linestyle='--', label=f'Optimal = {optimal_comp}')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('CV R²')
ax2.set_title('PLS Component Selection')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase1_pls_analysis.png', dpi=150, bbox_inches='tight')
plt.show()


# =============================================================================
# CELL 15: INTERACTION SCREENING
# =============================================================================

print("=" * 60)
print("INTERACTION SCREENING")
print("=" * 60)

top_n = min(6, len(feature_cols))
top_features = corr_df.head(top_n)['feature'].tolist()

interaction_results = []

for f1, f2 in combinations(top_features, 2):
    median_f2 = X[f2].median()
    low_f2 = X[f2] <= median_f2
    high_f2 = X[f2] > median_f2
    
    if low_f2.sum() >= 3 and high_f2.sum() >= 3:
        corr_low = X.loc[low_f2, f1].corr(y[low_f2])
        corr_high = X.loc[high_f2, f1].corr(y[high_f2])
        
        if not np.isnan(corr_low) and not np.isnan(corr_high):
            interaction_strength = abs(corr_high - corr_low)
            
            interaction_results.append({
                'interaction': f'{f1} × {f2}',
                'feature_1': f1,
                'feature_2': f2,
                'corr_low_f2': corr_low,
                'corr_high_f2': corr_high,
                'strength': interaction_strength,
                'significant': interaction_strength > INTERACTION_THRESHOLD
            })

if interaction_results:
    interaction_df = pd.DataFrame(interaction_results).sort_values('strength', ascending=False)
    
    print("\nInteraction Analysis:")
    print(interaction_df[['interaction', 'corr_low_f2', 'corr_high_f2', 'strength', 'significant']].to_string(index=False))
    
    strong_interactions = interaction_df[interaction_df['significant']]
    
    if len(strong_interactions) > 0:
        print(f"\n⚠️ POTENTIAL INTERACTIONS (strength > {INTERACTION_THRESHOLD}):")
        features_with_interactions = set()
        for _, row in strong_interactions.iterrows():
            print(f"  {row['interaction']}: {row['strength']:.3f}")
            features_with_interactions.add(row['feature_1'])
            features_with_interactions.add(row['feature_2'])
        features_with_interactions = list(features_with_interactions)
    else:
        print(f"\n✓ No strong interactions detected")
        features_with_interactions = []
    
    # Plot
    plt.figure(figsize=(10, 5))
    colors = ['crimson' if s else 'steelblue' for s in interaction_df['significant']]
    plt.barh(interaction_df['interaction'][::-1], interaction_df['strength'][::-1], color=colors[::-1])
    plt.axvline(x=INTERACTION_THRESHOLD, color='red', linestyle='--', linewidth=2)
    plt.xlabel('Interaction Strength')
    plt.title('Interaction Screening')
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'phase1_interactions.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("✓ No interactions to analyze")
    interaction_df = pd.DataFrame()
    strong_interactions = pd.DataFrame()
    features_with_interactions = []


# =============================================================================
# CELL 16: MULTICOLLINEARITY CHECK
# =============================================================================

print("=" * 60)
print("MULTICOLLINEARITY CHECK")
print("=" * 60)

feature_corr = X.corr()

high_corr_pairs = []
for i in range(len(feature_cols)):
    for j in range(i+1, len(feature_cols)):
        r = feature_corr.iloc[i, j]
        if abs(r) > MULTICOLLINEARITY_THRESHOLD:
            high_corr_pairs.append({
                'feature_1': feature_cols[i],
                'feature_2': feature_cols[j],
                'correlation': r
            })

if high_corr_pairs:
    print(f"⚠️ HIGHLY CORRELATED PAIRS (|r| > {MULTICOLLINEARITY_THRESHOLD}):")
    for pair in high_corr_pairs:
        print(f"  {pair['feature_1']} ↔ {pair['feature_2']}: r = {pair['correlation']:.3f}")
else:
    print("✓ No highly correlated feature pairs found")

# Heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(feature_corr, dtype=bool), k=0)
sns.heatmap(feature_corr, annot=True, cmap='RdBu_r', center=0, fmt='.2f',
            mask=mask, square=True, linewidths=0.5)
plt.title('Feature-Feature Correlations')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase1_multicollinearity.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# CELL 17: CONSENSUS RANKING
# =============================================================================

print("=" * 60)
print("CONSENSUS RANKING (All Methods)")
print("=" * 60)

consensus = corr_df[['feature', 'rank_corr', 'correlation', 'direction', 'type', 'strength']].merge(
    lasso_df[['feature', 'rank_lasso', 'coefficient', 'selected']], on='feature'
).merge(
    pls_df[['feature', 'rank_pls', 'VIP', 'category']], on='feature'
)

consensus['avg_rank'] = consensus[['rank_corr', 'rank_lasso', 'rank_pls']].mean(axis=1)
consensus = consensus.sort_values('avg_rank').reset_index(drop=True)
consensus['final_rank'] = range(1, len(consensus) + 1)

def count_top_k(row, k=3):
    count = 0
    if row['rank_corr'] <= k: count += 1
    if row['rank_lasso'] <= k: count += 1
    if row['rank_pls'] <= k: count += 1
    return count

consensus['methods_top3'] = consensus.apply(lambda r: count_top_k(r, 3), axis=1)
consensus['has_interaction'] = consensus['feature'].isin(features_with_interactions)

print("\nConsensus Ranking:")
display_cols = ['final_rank', 'feature', 'type', 'correlation', 'VIP', 'selected', 
                'avg_rank', 'methods_top3', 'has_interaction']
print(consensus[display_cols].to_string(index=False))


# =============================================================================
# CELL 18: CONSENSUS VISUALIZATION
# =============================================================================

print("=" * 60)
print("CONSENSUS VISUALIZATION")
print("=" * 60)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Heatmap of rankings
ax1 = axes[0, 0]
heatmap_data = consensus.set_index('feature')[['rank_corr', 'rank_lasso', 'rank_pls']]
heatmap_data.columns = ['Correlation', 'Lasso', 'PLS']
sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='RdYlGn_r', ax=ax1,
            cbar_kws={'label': 'Rank (lower=better)'})
ax1.set_title('Rankings Across Methods')

# Method agreement
ax2 = axes[0, 1]
colors = ['darkgreen' if a >= 3 else 'orange' if a >= 2 else 'lightcoral' 
          for a in consensus['methods_top3']]
ax2.barh(consensus['feature'][::-1], consensus['methods_top3'][::-1], color=colors[::-1])
ax2.axvline(x=2, color='orange', linestyle='--', linewidth=2)
ax2.set_xlabel('Methods Ranking Feature in Top 3')
ax2.set_title('Method Agreement')

# Average rank
ax3 = axes[1, 0]
colors = ['steelblue' if t == 'binary' else 'forestgreen' for t in consensus['type']]
ax3.barh(consensus['feature'][::-1], consensus['avg_rank'][::-1], color=colors[::-1])
ax3.set_xlabel('Average Rank (lower = better)')
ax3.set_title('Consensus Ranking')
ax3.invert_xaxis()

# Correlation vs VIP
ax4 = axes[1, 1]
for _, row in consensus.iterrows():
    color = 'steelblue' if row['type'] == 'binary' else 'forestgreen'
    marker = 's' if row['has_interaction'] else 'o'
    ax4.scatter(abs(row['correlation']), row['VIP'], c=color, s=100, marker=marker, 
                edgecolors='black', linewidth=0.5)
    ax4.annotate(row['feature'], (abs(row['correlation']), row['VIP']), 
                 fontsize=8, ha='left', va='bottom')

ax4.axhline(y=VIP_IMPORTANT, color='green', linestyle='--', alpha=0.7)
ax4.axvline(x=CORRELATION_STRONG, color='blue', linestyle='--', alpha=0.7)
ax4.set_xlabel('|Correlation|')
ax4.set_ylabel('VIP Score')
ax4.set_title('Correlation vs VIP')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase1_consensus.png', dpi=150, bbox_inches='tight')
plt.show()


# =============================================================================
# CELL 19: AUTOMATIC FEATURE RECOMMENDATION
# =============================================================================

print("=" * 60)
print("FEATURE RECOMMENDATION")
print("=" * 60)

def score_feature(row):
    score = 0
    if abs(row['correlation']) >= CORRELATION_STRONG: score += 3
    elif abs(row['correlation']) >= CORRELATION_MODERATE: score += 2
    if row['VIP'] >= VIP_IMPORTANT: score += 3
    elif row['VIP'] >= VIP_MODERATE: score += 2
    if row['selected']: score += 2
    score += row['methods_top3']
    if row['has_interaction']: score += 2
    return score

consensus['score'] = consensus.apply(score_feature, axis=1)
consensus = consensus.sort_values('score', ascending=False).reset_index(drop=True)

print("\nFeature Scores:")
print(consensus[['feature', 'type', 'correlation', 'VIP', 'selected', 'has_interaction', 'score']].to_string(index=False))

# Generate recommendations
recommended = []
reasons = {}

for _, row in consensus.iterrows():
    include = False
    reason = []
    
    if row['score'] >= 6:
        include = True
        reason.append(f"High score ({row['score']})")
    if abs(row['correlation']) >= CORRELATION_STRONG:
        include = True
        reason.append("Strong correlation")
    if row['VIP'] >= VIP_IMPORTANT:
        include = True
        reason.append(f"VIP ≥ {VIP_IMPORTANT}")
    if row['has_interaction'] and row['score'] >= 4:
        include = True
        reason.append("Part of interaction")
    
    if include and len(recommended) < 6:
        recommended.append(row['feature'])
        reasons[row['feature']] = ', '.join(reason)

# Ensure minimum
MIN_FEATURES = 3
if len(recommended) < MIN_FEATURES:
    for _, row in consensus.iterrows():
        if row['feature'] not in recommended:
            recommended.append(row['feature'])
            reasons[row['feature']] = "Added to meet minimum"
        if len(recommended) >= MIN_FEATURES:
            break

print(f"\n{'='*50}")
print(f"RECOMMENDED ({len(recommended)} features):")
print(f"{'='*50}")

for feat in recommended:
    row = consensus[consensus['feature'] == feat].iloc[0]
    print(f"\n  ✓ {feat} ({row['type']})")
    print(f"      Correlation: {row['correlation']:.3f}")
    print(f"      VIP: {row['VIP']:.2f}")
    print(f"      Reason: {reasons[feat]}")


# =============================================================================
# CELL 20: MANUAL FEATURE SELECTION
# =============================================================================

print("=" * 60)
print("FEATURE SELECTION")
print("=" * 60)

# ┌─────────────────────────────────────────────────────────────────────────┐
# │ OPTION 1: Use automatic recommendations (DEFAULT)                       │
# └─────────────────────────────────────────────────────────────────────────┘
selected_features = recommended[:TARGET_FEATURES]

# ┌─────────────────────────────────────────────────────────────────────────┐
# │ OPTION 2: Manual override - UNCOMMENT AND MODIFY IF NEEDED             │
# └─────────────────────────────────────────────────────────────────────────┘
# selected_features = [
#     'Your_Feature_1',  # Keep from recommendations
#     'Your_Feature_2',  # Keep from recommendations
#     'Your_Feature_3',  # Keep from recommendations
#     'Your_New_Feature', # YOUR MANUAL OVERRIDE - replace one you don't like
# ]

print(f"\nSelected features ({len(selected_features)}):")
for i, feat in enumerate(selected_features, 1):
    row = consensus[consensus['feature'] == feat].iloc[0]
    print(f"  {i}. {feat} ({row['type']}) - corr: {row['correlation']:.3f}, VIP: {row['VIP']:.2f}")

# Validation checks
print("\n" + "-" * 40)
print("VALIDATION CHECKS:")

# Check multicollinearity
if len(selected_features) > 1:
    sel_corr = X[selected_features].corr()
    issues = []
    for i in range(len(selected_features)):
        for j in range(i+1, len(selected_features)):
            r = sel_corr.iloc[i, j]
            if abs(r) > MULTICOLLINEARITY_THRESHOLD:
                issues.append(f"{selected_features[i]} ↔ {selected_features[j]}: r={r:.2f}")
    
    if issues:
        print(f"  ⚠️ Multicollinearity detected:")
        for issue in issues:
            print(f"      {issue}")
    else:
        print(f"  ✓ No multicollinearity issues")

# Check broken interactions
if len(strong_interactions) > 0:
    broken = []
    for _, row in strong_interactions.iterrows():
        f1, f2 = row['feature_1'], row['feature_2']
        if (f1 in selected_features) != (f2 in selected_features):
            broken.append((f1, f2))
    
    if broken:
        print(f"  ⚠️ Broken interactions:")
        for f1, f2 in broken:
            in1 = "IN" if f1 in selected_features else "OUT"
            in2 = "IN" if f2 in selected_features else "OUT"
            print(f"      {f1} ({in1}) × {f2} ({in2})")
    else:
        print(f"  ✓ No broken interactions")
else:
    print(f"  ✓ No interactions to check")


# =============================================================================
# CELL 21: VALIDATE WITH LOO-CV (LEAKAGE-FREE)
# =============================================================================

print("=" * 60)
print("VALIDATION: Leave-One-Out Cross-Validation (Leakage-Free)")
print("=" * 60)

# Prepare selected feature data
X_sel = X[selected_features].copy()

# Identify which selected features are continuous
sel_continuous = [f for f in selected_features if f in CONTINUOUS_FEATURES]
sel_binary = [f for f in selected_features if f in BINARY_FEATURES]

# LOO-CV with proper scaling inside the loop (NO LEAKAGE)
loo_preds = []
loo_actual = []

for train_idx, test_idx in LeaveOneOut().split(X_sel):
    X_train = X_sel.iloc[train_idx].copy()
    X_test = X_sel.iloc[test_idx].copy()
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]
    
    # Scale ONLY on training data
    if sel_continuous:
        scaler_cv = StandardScaler()
        X_train[sel_continuous] = scaler_cv.fit_transform(X_train[sel_continuous])
        X_test[sel_continuous] = scaler_cv.transform(X_test[sel_continuous])
    
    # Convert binary to [-1, +1]
    for col in sel_binary:
        X_train[col] = X_train[col] * 2 - 1
        X_test[col] = X_test[col] * 2 - 1
    
    # Fit model
    model = RidgeCV(alphas=[0.1, 1, 10, 100], cv=3)
    model.fit(X_train, y_train)
    
    # Predict
    loo_preds.append(model.predict(X_test)[0])
    loo_actual.append(y_test.values[0])

loo_preds = np.array(loo_preds)
loo_actual = np.array(loo_actual)

# Metrics
loo_r2 = r2_score(loo_actual, loo_preds)
loo_rmse = np.sqrt(mean_squared_error(loo_actual, loo_preds))
loo_mae = np.mean(np.abs(loo_actual - loo_preds))

print(f"\nLOO-CV Results ({len(selected_features)} features):")
print(f"  R²:   {loo_r2:.4f}")
print(f"  RMSE: {loo_rmse:.4f}")
print(f"  MAE:  {loo_mae:.4f}")

print(f"\nInterpretation:")
if loo_r2 > 0.5:
    print("  ✓ Good signal - features are predictive")
elif loo_r2 > 0.2:
    print("  ⚠️ Moderate signal - GP in BO can likely improve")
elif loo_r2 > 0:
    print("  ⚠️ Weak linear signal - may be non-linear relationships")
else:
    print("  ⚠️ No linear signal - review feature selection")

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

ax1 = axes[0]
ax1.scatter(loo_actual, loo_preds, alpha=0.7, edgecolors='black', linewidth=0.5)
lims = [min(loo_actual.min(), loo_preds.min()), max(loo_actual.max(), loo_preds.max())]
ax1.plot(lims, lims, 'r--', linewidth=2, label='Perfect')
ax1.set_xlabel('Actual')
ax1.set_ylabel('Predicted')
ax1.set_title(f'LOO-CV: R² = {loo_r2:.4f}')
ax1.legend()

ax2 = axes[1]
residuals = loo_actual - loo_preds
ax2.hist(residuals, bins=12, edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Residual')
ax2.set_ylabel('Frequency')
ax2.set_title(f'Residuals: Mean={residuals.mean():.3f}')

ax3 = axes[2]
ax3.scatter(loo_preds, residuals, alpha=0.7, edgecolors='black', linewidth=0.5)
ax3.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('Predicted')
ax3.set_ylabel('Residual')
ax3.set_title('Residuals vs Predicted')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase1_validation.png', dpi=150, bbox_inches='tight')
plt.show()


# =============================================================================
# CELL 22: DEFINE BO SEARCH SPACE
# =============================================================================

print("=" * 60)
print("BAYESIAN OPTIMIZATION SEARCH SPACE")
print("=" * 60)

bo_bounds = []
for feat in selected_features:
    if feat in BINARY_FEATURES:
        bo_bounds.append({
            'feature': feat,
            'type': 'binary',
            'min': 0,
            'max': 1,
            'observed_min': 0,
            'observed_max': 1
        })
    else:
        feat_min = X_original[feat].min()
        feat_max = X_original[feat].max()
        feat_range = feat_max - feat_min
        margin = 0.1 * feat_range
        bo_bounds.append({
            'feature': feat,
            'type': 'continuous',
            'min': round(feat_min - margin, 4),
            'max': round(feat_max + margin, 4),
            'observed_min': feat_min,
            'observed_max': feat_max
        })

bo_bounds_df = pd.DataFrame(bo_bounds)

print("\nSearch Space Bounds:")
print(bo_bounds_df.to_string(index=False))

print("\n" + "-" * 40)
print("EFFECT DIRECTIONS:")
for feat in selected_features:
    row = consensus[consensus['feature'] == feat].iloc[0]
    direction = row['correlation']
    
    if MAXIMIZE_RESPONSE:
        suggest = "HIGH" if direction > 0 else "LOW"
    else:
        suggest = "LOW" if direction > 0 else "HIGH"
    
    print(f"  {feat}: {'Positive' if direction > 0 else 'Negative'} effect → Suggest {suggest}")


# =============================================================================
# CELL 23: CREATE COMPREHENSIVE FEATURE INFO
# =============================================================================

print("=" * 60)
print("COMPILING FEATURE INFORMATION")
print("=" * 60)

feature_info = []

for feat in selected_features:
    row = consensus[consensus['feature'] == feat].iloc[0]
    bounds_row = bo_bounds_df[bo_bounds_df['feature'] == feat].iloc[0]
    
    info = {
        'feature': feat,
        'type': row['type'],
        'correlation': row['correlation'],
        'correlation_strength': row['strength'],
        'direction': row['direction'],
        'lasso_coefficient': row['coefficient'],
        'lasso_selected': row['selected'],
        'vip_score': row['VIP'],
        'vip_category': row['category'],
        'consensus_rank': int(row['final_rank']),
        'methods_top3': int(row['methods_top3']),
        'has_interaction': row['has_interaction'],
        'feature_score': int(row['score']),
        'bound_min': bounds_row['min'],
        'bound_max': bounds_row['max'],
        'observed_min': bounds_row['observed_min'],
        'observed_max': bounds_row['observed_max'],
    }
    
    # Add binary mapping if applicable
    if feat in binary_mappings:
        mapping = binary_mappings[feat]
        info['binary_value_0'] = list(mapping.keys())[0]
        info['binary_value_1'] = list(mapping.keys())[1]
    else:
        info['binary_value_0'] = None
        info['binary_value_1'] = None
    
    feature_info.append(info)

feature_info_df = pd.DataFrame(feature_info)

print("\nSelected Feature Details:")
print(feature_info_df.to_string(index=False))


# =============================================================================
# CELL 24: SAVE PHASE 1 CHECKPOINT
# =============================================================================

print("=" * 60)
print("SAVING PHASE 1 CHECKPOINT")
print("=" * 60)

checkpoint_time = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create checkpoint dictionary with all relevant information
phase1_checkpoint = {
    'metadata': {
        'checkpoint_time': checkpoint_time,
        'phase1_data_file': PHASE1_DATA_FILE,
        'phase1_sheet_name': PHASE1_SHEET_NAME,
        'response_column': RESPONSE_COLUMN,
        'maximize_response': MAXIMIZE_RESPONSE,
        'n_initial_experiments': len(X),
        'n_total_features': len(feature_cols),
        'n_selected_features': len(selected_features),
        'target_features': TARGET_FEATURES,
    },
    'thresholds': {
        'correlation_strong': CORRELATION_STRONG,
        'correlation_moderate': CORRELATION_MODERATE,
        'vip_important': VIP_IMPORTANT,
        'vip_moderate': VIP_MODERATE,
        'multicollinearity_threshold': MULTICOLLINEARITY_THRESHOLD,
        'interaction_threshold': INTERACTION_THRESHOLD,
    },
    'selected_features': selected_features,
    'all_features': feature_cols,
    'binary_features': BINARY_FEATURES,
    'continuous_features': CONTINUOUS_FEATURES,
    'selected_binary': [f for f in selected_features if f in BINARY_FEATURES],
    'selected_continuous': [f for f in selected_features if f in CONTINUOUS_FEATURES],
    'binary_mappings': binary_mappings,
    'validation_metrics': {
        'loo_r2': float(loo_r2),
        'loo_rmse': float(loo_rmse),
        'loo_mae': float(loo_mae),
    },
    'response_stats': {
        'min': float(y.min()),
        'max': float(y.max()),
        'mean': float(y.mean()),
        'median': float(y.median()),
        'std': float(y.std()),
    },
    'interactions': {
        'features_with_interactions': features_with_interactions,
        'strong_interactions': strong_interactions.to_dict('records') if len(strong_interactions) > 0 else [],
    },
    'multicollinearity': {
        'high_correlation_pairs': high_corr_pairs,
    },
}

# Save as JSON (human-readable)
checkpoint_json_path = OUTPUT_DIR / 'phase1_checkpoint.json'
with open(checkpoint_json_path, 'w') as f:
    json.dump(phase1_checkpoint, f, indent=2, default=str)
print(f"✓ Saved: {checkpoint_json_path}")

# Save as pickle (preserves all Python objects)
checkpoint_pkl_path = OUTPUT_DIR / 'phase1_checkpoint.pkl'
with open(checkpoint_pkl_path, 'wb') as f:
    pickle.dump(phase1_checkpoint, f)
print(f"✓ Saved: {checkpoint_pkl_path}")

# Save feature info DataFrame
feature_info_path = OUTPUT_DIR / 'phase1_feature_info.csv'
feature_info_df.to_csv(feature_info_path, index=False)
print(f"✓ Saved: {feature_info_path}")

# Save bounds DataFrame
bounds_path = OUTPUT_DIR / 'phase1_bounds.csv'
bo_bounds_df.to_csv(bounds_path, index=False)
print(f"✓ Saved: {bounds_path}")

# Save consensus ranking (all features)
consensus_path = OUTPUT_DIR / 'phase1_consensus_ranking.csv'
consensus.to_csv(consensus_path, index=False)
print(f"✓ Saved: {consensus_path}")

# Save original Phase 1 data (for reference)
phase1_data_path = OUTPUT_DIR / 'phase1_original_data.csv'
df_phase1.to_csv(phase1_data_path, index=False)
print(f"✓ Saved: {phase1_data_path}")

print("\n" + "=" * 60)
print("CHECKPOINT CONTENTS SUMMARY")
print("=" * 60)
print(f"""
Files saved to: {OUTPUT_DIR}/

1. phase1_checkpoint.json    - Human-readable checkpoint
2. phase1_checkpoint.pkl     - Python pickle checkpoint
3. phase1_feature_info.csv   - Selected feature details
4. phase1_bounds.csv         - BO search space bounds
5. phase1_consensus_ranking.csv - All features ranked
6. phase1_original_data.csv  - Original experiment data
7. Various PNG plots         - Visualization exports
""")


# =============================================================================
# CELL 25: PHASE 1 SUMMARY
# =============================================================================

print("=" * 70)
print("PHASE 1 COMPLETE: FEATURE SCREENING SUMMARY")
print("=" * 70)

print(f"""
DATA ANALYZED:
  Total samples: {len(X)}
  Total features: {len(feature_cols)}
    - Binary: {len(BINARY_FEATURES)}
    - Continuous: {len(CONTINUOUS_FEATURES)}

METHODS USED:
  1. Pearson Correlation Analysis
  2. Lasso Regression (automatic selection)
  3. PLS with VIP Scores
  4. Interaction Screening
  5. Multicollinearity Check

SELECTED FOR BAYESIAN OPTIMIZATION ({len(selected_features)} features):
""")

for feat in selected_features:
    row = consensus[consensus['feature'] == feat].iloc[0]
    bounds_row = bo_bounds_df[bo_bounds_df['feature'] == feat].iloc[0]
    int_flag = " ⚡" if row['has_interaction'] else ""
    print(f"  • {feat} ({row['type']}){int_flag}")
    print(f"      Correlation: {row['correlation']:+.3f} ({row['strength']})")
    print(f"      VIP Score: {row['VIP']:.2f} ({row['category']})")
    print(f"      Lasso: {'Selected' if row['selected'] else 'Not selected'}")
    print(f"      Bounds: [{bounds_row['min']:.4f}, {bounds_row['max']:.4f}]")

if features_with_interactions:
    print(f"\n  ⚡ = Part of detected interaction")

print(f"""
VALIDATION (Leakage-Free LOO-CV):
  R²:   {loo_r2:.4f}
  RMSE: {loo_rmse:.4f}
  MAE:  {loo_mae:.4f}

CHECKPOINT SAVED:
  Location: {OUTPUT_DIR}/
  Use phase1_checkpoint.json or .pkl to load settings for Phase 2
""")


# =============================================================================
# CELL 26: INSTRUCTIONS FOR NEXT STEPS
# =============================================================================

print("=" * 70)
print("NEXT STEPS: PREPARE FOR PHASE 2")
print("=" * 70)

print(f"""
┌─────────────────────────────────────────────────────────────────────────┐
│ WHAT TO DO NOW:                                                         │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│ 1. REVIEW SELECTED FEATURES                                             │
│    Current selection: {selected_features}
│                                                                         │
│ 2. IF YOU WANT TO CHANGE A FEATURE:                                     │
│    → Go back to Cell 20                                                 │
│    → Uncomment the manual override section                              │
│    → Modify the list                                                    │
│    → Re-run Cells 20-25 to update checkpoint                           │
│                                                                         │
│ 3. DESIGN NEW EXPERIMENTS                                               │
│    → Use ONLY the {len(selected_features)} selected features                                     │
│    → Recommended: 30 experiments                                        │
│    → Design type: Latin Hypercube or space-filling                     │
│    → Use bounds from phase1_bounds.csv                                 │
│                                                                         │
│ 4. RUN EXPERIMENTS IN LAB                                               │
│    → Execute the 30 new experiments                                    │
│    → Record results in Excel                                           │
│                                                                         │
│ 5. PREPARE PHASE 2 DATA FILE                                            │
│    → Create Excel/CSV with columns:                                    │
│      {selected_features + [RESPONSE_COLUMN]}
│    → Save as 'phase2_experiments.xlsx' (or update config)             │
│                                                                         │
│ 6. CONTINUE TO PHASE 2                                                  │
│    → Run cells starting from Cell 27                                   │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
""")

print("=" * 70)
print("END OF PHASE 1 - PROCEED TO PHASE 2 WHEN NEW DATA IS READY")
print("=" * 70)


# =============================================================================
# =============================================================================
# PHASE 2: BAYESIAN OPTIMIZATION
# =============================================================================
# =============================================================================

# ┌─────────────────────────────────────────────────────────────────────────┐
# │ RUN CELLS BELOW ONLY AFTER:                                            │
# │   1. You have selected your final features (Phase 1)                   │
# │   2. You have run new experiments with those features                  │
# │   3. You have created a new data file with results                     │
# └─────────────────────────────────────────────────────────────────────────┘


# =============================================================================
# CELL 27: PHASE 2 CONFIGURATION
# =============================================================================

print("\n" + "=" * 70)
print("PHASE 2: BAYESIAN OPTIMIZATION")
print("=" * 70)

# ┌─────────────────────────────────────────────────────────────────────────┐
# │ MODIFY THIS SECTION FOR YOUR NEW DATA                                   │
# └─────────────────────────────────────────────────────────────────────────┘

# Phase 2 data file (your NEW experiments with selected features only)
PHASE2_DATA_FILE = 'phase2_experiments.xlsx'  # UPDATE THIS
PHASE2_SHEET_NAME = 'Sheet1'                   # UPDATE THIS

# BO Settings
BATCH_SIZE = 5                    # Number of experiments per iteration
EXPLORATION_WEIGHT = 2.0          # Kappa for UCB (higher = more exploration)
N_OPTIMIZER_RESTARTS = 20         # Restarts for acquisition optimization
MIN_DISTANCE_BETWEEN_POINTS = 0.3 # Minimum distance between batch points (scaled)

# Convergence settings
MAX_ITERATIONS = 10               # Maximum BO iterations
IMPROVEMENT_THRESHOLD = 0.01      # Stop if improvement < this for N iterations
PATIENCE = 3                      # Number of iterations without improvement before stopping

print("Phase 2 Configuration:")
print(f"  Data file: {PHASE2_DATA_FILE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Exploration weight (κ): {EXPLORATION_WEIGHT}")
print(f"  Max iterations: {MAX_ITERATIONS}")


# =============================================================================
# CELL 28: LOAD PHASE 1 CHECKPOINT
# =============================================================================

print("=" * 60)
print("LOADING PHASE 1 CHECKPOINT")
print("=" * 60)

# Load checkpoint
checkpoint_path = OUTPUT_DIR / 'phase1_checkpoint.pkl'

if checkpoint_path.exists():
    with open(checkpoint_path, 'rb') as f:
        phase1_checkpoint = pickle.load(f)
    print(f"✓ Loaded checkpoint from: {checkpoint_path}")
else:
    raise FileNotFoundError(f"Phase 1 checkpoint not found at {checkpoint_path}. Run Phase 1 first.")

# Extract key information
SELECTED_FEATURES = phase1_checkpoint['selected_features']
BINARY_FEATURES_SELECTED = phase1_checkpoint['selected_binary']
CONTINUOUS_FEATURES_SELECTED = phase1_checkpoint['selected_continuous']
RESPONSE_COLUMN = phase1_checkpoint['metadata']['response_column']
MAXIMIZE_RESPONSE = phase1_checkpoint['metadata']['maximize_response']
BINARY_MAPPINGS = phase1_checkpoint['binary_mappings']

print(f"\nRestored settings:")
print(f"  Selected features: {SELECTED_FEATURES}")
print(f"    Binary: {BINARY_FEATURES_SELECTED}")
print(f"    Continuous: {CONTINUOUS_FEATURES_SELECTED}")
print(f"  Response: {RESPONSE_COLUMN}")
print(f"  Objective: {'Maximize' if MAXIMIZE_RESPONSE else 'Minimize'}")

# Load bounds
bounds_df = pd.read_csv(OUTPUT_DIR / 'phase1_bounds.csv')
print(f"\n✓ Loaded bounds:")
print(bounds_df.to_string(index=False))

# Load feature info
feature_info_df = pd.read_csv(OUTPUT_DIR / 'phase1_feature_info.csv')
print(f"\n✓ Loaded feature info")


# =============================================================================
# CELL 29: LOAD PHASE 2 DATA (NEW EXPERIMENTS)
# =============================================================================

print("=" * 60)
print("LOADING PHASE 2 DATA")
print("=" * 60)

# Load new experiment data
try:
    if PHASE2_DATA_FILE.endswith('.csv'):
        df_phase2 = pd.read_csv(PHASE2_DATA_FILE)
    else:
        df_phase2 = pd.read_excel(PHASE2_DATA_FILE, sheet_name=PHASE2_SHEET_NAME)
    print(f"✓ Loaded: {PHASE2_DATA_FILE}")
    print(f"  Shape: {df_phase2.shape}")
except FileNotFoundError:
    print(f"⚠️ File not found: {PHASE2_DATA_FILE}")
    print(f"   Please create this file with your new experiment results.")
    print(f"   Required columns: {SELECTED_FEATURES + [RESPONSE_COLUMN]}")
    raise

# Validate columns
missing_cols = [c for c in SELECTED_FEATURES + [RESPONSE_COLUMN] if c not in df_phase2.columns]
if missing_cols:
    raise ValueError(f"Missing columns in Phase 2 data: {missing_cols}")

print(f"✓ All required columns present")

# Extract X and y
X_phase2 = df_phase2[SELECTED_FEATURES].copy()
y_phase2 = df_phase2[RESPONSE_COLUMN].copy()

# Handle missing values
valid = ~y_phase2.isnull() & X_phase2.notna().all(axis=1)
n_dropped = (~valid).sum()
X_phase2 = X_phase2[valid].reset_index(drop=True)
y_phase2 = y_phase2[valid].reset_index(drop=True)

if n_dropped > 0:
    print(f"  Dropped {n_dropped} rows with missing values")

print(f"\n  Valid experiments: {len(X_phase2)}")

# Store original data
X_original_phase2 = X_phase2.copy()
y_original_phase2 = y_phase2.copy()


# =============================================================================
# CELL 30: VALIDATE PHASE 2 DATA
# =============================================================================

print("=" * 60)
print("PHASE 2 DATA VALIDATION")
print("=" * 60)

print("\nFeature Ranges:")
for feat in SELECTED_FEATURES:
    bounds_row = bounds_df[bounds_df['feature'] == feat].iloc[0]
    actual_min = X_phase2[feat].min()
    actual_max = X_phase2[feat].max()
    expected_min = bounds_row['min']
    expected_max = bounds_row['max']
    
    in_bounds = (actual_min >= expected_min * 0.9) and (actual_max <= expected_max * 1.1)
    status = "✓" if in_bounds else "⚠️"
    
    print(f"  {status} {feat}:")
    print(f"      Actual: [{actual_min:.4f}, {actual_max:.4f}]")
    print(f"      Expected: [{expected_min:.4f}, {expected_max:.4f}]")

print(f"\nResponse ({RESPONSE_COLUMN}):")
print(f"  Min:  {y_phase2.min():.4f}")
print(f"  Max:  {y_phase2.max():.4f}")
print(f"  Mean: {y_phase2.mean():.4f}")
print(f"  Std:  {y_phase2.std():.4f}")

# Compare with Phase 1 response stats
phase1_stats = phase1_checkpoint['response_stats']
print(f"\nComparison with Phase 1:")
print(f"  Phase 1 range: [{phase1_stats['min']:.4f}, {phase1_stats['max']:.4f}]")
print(f"  Phase 2 range: [{y_phase2.min():.4f}, {y_phase2.max():.4f}]")


# =============================================================================
# CELL 31: PREPARE BOUNDS ARRAY
# =============================================================================

print("=" * 60)
print("PREPARING SEARCH BOUNDS")
print("=" * 60)

bounds_dict = {}
bounds_array = []

for feat in SELECTED_FEATURES:
    bounds_row = bounds_df[bounds_df['feature'] == feat].iloc[0]
    lb = bounds_row['min']
    ub = bounds_row['max']
    bounds_dict[feat] = (lb, ub)
    bounds_array.append([lb, ub])

bounds_array = np.array(bounds_array)

print("Search bounds (original scale):")
for feat in SELECTED_FEATURES:
    lb, ub = bounds_dict[feat]
    print(f"  {feat}: [{lb:.4f}, {ub:.4f}]")


# =============================================================================
# CELL 32: DEFINE SCALING FUNCTIONS
# =============================================================================

print("=" * 60)
print("DEFINING SCALING FUNCTIONS")
print("=" * 60)

class Phase2Scaler:
    """
    Scaler that handles both continuous standardization and binary encoding.
    Fits only on training data to prevent leakage.
    """
    
    def __init__(self, continuous_features, binary_features, all_features):
        self.continuous_features = continuous_features
        self.binary_features = binary_features
        self.all_features = all_features
        self.scaler = StandardScaler()
        self.is_fitted = False
        
    def fit(self, X):
        """Fit scaler on training data only."""
        if self.continuous_features:
            self.scaler.fit(X[self.continuous_features])
        self.is_fitted = True
        return self
    
    def transform(self, X):
        """Transform data using fitted scaler."""
        if not self.is_fitted:
            raise ValueError("Scaler not fitted. Call fit() first.")
        
        X_transformed = X.copy()
        
        # Scale continuous features
        if self.continuous_features:
            X_transformed[self.continuous_features] = self.scaler.transform(X[self.continuous_features])
        
        # Convert binary to [-1, +1]
        for col in self.binary_features:
            X_transformed[col] = X_transformed[col] * 2 - 1
        
        return X_transformed
    
    def fit_transform(self, X):
        """Fit and transform in one step."""
        return self.fit(X).transform(X)
    
    def inverse_transform_point(self, x_scaled):
        """Convert a single scaled point back to original space."""
        x_original = np.array(x_scaled).copy()
        
        for i, feat in enumerate(self.all_features):
            if feat in self.binary_features:
                # Convert [-1, +1] back to [0, 1]
                x_original[i] = (x_original[i] + 1) / 2
            elif feat in self.continuous_features:
                # Inverse standardization
                cont_idx = self.continuous_features.index(feat)
                x_original[i] = x_original[i] * self.scaler.scale_[cont_idx] + self.scaler.mean_[cont_idx]
        
        return x_original
    
    def transform_point(self, x_original):
        """Convert a single original point to scaled space."""
        x_scaled = np.array(x_original).copy()
        
        for i, feat in enumerate(self.all_features):
            if feat in self.binary_features:
                # Convert [0, 1] to [-1, +1]
                x_scaled[i] = x_scaled[i] * 2 - 1
            elif feat in self.continuous_features:
                # Standardization
                cont_idx = self.continuous_features.index(feat)
                x_scaled[i] = (x_scaled[i] - self.scaler.mean_[cont_idx]) / self.scaler.scale_[cont_idx]
        
        return x_scaled
    
    def get_scaled_bounds(self, bounds_array):
        """Convert bounds to scaled space."""
        scaled_bounds = []
        
        for i, feat in enumerate(self.all_features):
            if feat in self.binary_features:
                scaled_bounds.append([-1, 1])
            else:
                cont_idx = self.continuous_features.index(feat)
                lb_scaled = (bounds_array[i, 0] - self.scaler.mean_[cont_idx]) / self.scaler.scale_[cont_idx]
                ub_scaled = (bounds_array[i, 1] - self.scaler.mean_[cont_idx]) / self.scaler.scale_[cont_idx]
                scaled_bounds.append([lb_scaled, ub_scaled])
        
        return np.array(scaled_bounds)

print("✓ Phase2Scaler class defined")
print("  Handles continuous standardization and binary encoding")
print("  Prevents data leakage by fitting only on training data")


# =============================================================================
# CELL 33: DEFINE ACQUISITION FUNCTIONS
# =============================================================================

print("=" * 60)
print("DEFINING ACQUISITION FUNCTIONS")
print("=" * 60)

def expected_improvement(X_new, gp, y_best, xi=0.01, minimize=True):
    """
    Expected Improvement acquisition function.
    
    Args:
        X_new: Points to evaluate (n_points, n_features)
        gp: Fitted GP model
        y_best: Best observed value
        xi: Exploration-exploitation trade-off parameter
        minimize: If True, we want to find minimum
    
    Returns:
        EI values (n_points,)
    """
    X_new = np.atleast_2d(X_new)
    mu, sigma = gp.predict(X_new, return_std=True)
    sigma = np.maximum(sigma, 1e-8)
    
    if minimize:
        improvement = y_best - mu - xi
    else:
        improvement = mu - y_best - xi
    
    Z = improvement / sigma
    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-8] = 0.0
    
    return ei


def lower_confidence_bound(X_new, gp, kappa=2.0):
    """
    Lower Confidence Bound for minimization.
    
    Args:
        X_new: Points to evaluate
        gp: Fitted GP model
        kappa: Exploration weight (higher = more exploration)
    
    Returns:
        LCB values (lower is better for minimization)
    """
    X_new = np.atleast_2d(X_new)
    mu, sigma = gp.predict(X_new, return_std=True)
    return mu - kappa * sigma


def upper_confidence_bound(X_new, gp, kappa=2.0):
    """
    Upper Confidence Bound for maximization.
    
    Args:
        X_new: Points to evaluate
        gp: Fitted GP model
        kappa: Exploration weight (higher = more exploration)
    
    Returns:
        UCB values (higher is better for maximization)
    """
    X_new = np.atleast_2d(X_new)
    mu, sigma = gp.predict(X_new, return_std=True)
    return mu + kappa * sigma


def probability_of_improvement(X_new, gp, y_best, xi=0.01, minimize=True):
    """
    Probability of Improvement acquisition function.
    """
    X_new = np.atleast_2d(X_new)
    mu, sigma = gp.predict(X_new, return_std=True)
    sigma = np.maximum(sigma, 1e-8)
    
    if minimize:
        Z = (y_best - mu - xi) / sigma
    else:
        Z = (mu - y_best - xi) / sigma
    
    return norm.cdf(Z)


print("✓ Acquisition functions defined:")
print("  • Expected Improvement (EI)")
print("  • Lower Confidence Bound (LCB) - for minimization")
print("  • Upper Confidence Bound (UCB) - for maximization")
print("  • Probability of Improvement (PI)")


# =============================================================================
# CELL 34: DEFINE BATCH SELECTION FUNCTION
# =============================================================================

print("=" * 60)
print("DEFINING BATCH SELECTION")
print("=" * 60)

def optimize_acquisition_batch(gp, bounds_scaled, n_batch, y_best, minimize=True, 
                                kappa=2.0, n_restarts=20, min_distance=0.3):
    """
    Find a batch of diverse points that optimize the acquisition function.
    
    Uses greedy selection with diversity constraint.
    """
    
    def negative_lcb(x):
        """Negative LCB for scipy minimization."""
        x = x.reshape(1, -1)
        if minimize:
            return lower_confidence_bound(x, gp, kappa)[0]
        else:
            return -upper_confidence_bound(x, gp, kappa)[0]
    
    def negative_ei(x):
        """Negative EI for scipy minimization."""
        x = x.reshape(1, -1)
        return -expected_improvement(x, gp, y_best, minimize=minimize)[0]
    
    n_features = bounds_scaled.shape[0]
    selected_points = []
    selected_acq_values = []
    
    for batch_idx in range(n_batch):
        best_candidates = []
        
        # Multi-start optimization
        for _ in range(n_restarts):
            # Random starting point
            x0 = np.random.uniform(bounds_scaled[:, 0], bounds_scaled[:, 1])
            
            try:
                # Use EI for first point, LCB for diversity
                if batch_idx == 0:
                    result = minimize(negative_ei, x0, method='L-BFGS-B', 
                                     bounds=bounds_scaled)
                else:
                    result = minimize(negative_lcb, x0, method='L-BFGS-B',
                                     bounds=bounds_scaled)
                
                if result.success:
                    # Check distance to already selected points
                    if selected_points:
                        distances = [np.linalg.norm(result.x - p) for p in selected_points]
                        if min(distances) >= min_distance:
                            best_candidates.append((result.fun, result.x))
                    else:
                        best_candidates.append((result.fun, result.x))
            except:
                continue
        
        # Select best candidate
        if best_candidates:
            best_candidates.sort(key=lambda x: x[0])
            selected_points.append(best_candidates[0][1])
            selected_acq_values.append(best_candidates[0][0])
        else:
            # Fallback: random point with distance constraint
            for _ in range(100):
                x_random = np.random.uniform(bounds_scaled[:, 0], bounds_scaled[:, 1])
                if not selected_points or min(np.linalg.norm(x_random - p) for p in selected_points) >= min_distance:
                    selected_points.append(x_random)
                    selected_acq_values.append(np.inf)
                    break
    
    return np.array(selected_points), np.array(selected_acq_values)

print("✓ Batch selection function defined")
print("  Uses greedy selection with diversity constraints")


# =============================================================================
# CELL 35: INITIALIZE BO TRACKING
# =============================================================================

print("=" * 60)
print("INITIALIZING BAYESIAN OPTIMIZATION")
print("=" * 60)

# Initialize tracking variables
bo_history = {
    'iterations': [],
    'best_values': [],
    'proposed_points': [],
    'observed_responses': [],
    'gp_params': [],
}

# Current data
X_current = X_phase2.copy()
y_current = y_phase2.copy()

# Current best
if MAXIMIZE_RESPONSE:
    current_best = y_current.max()
    current_best_idx = y_current.idxmax()
else:
    current_best = y_current.min()
    current_best_idx = y_current.idxmin()

print(f"\nInitial state:")
print(f"  Training samples: {len(X_current)}")
print(f"  Current best {'maximum' if MAXIMIZE_RESPONSE else 'minimum'}: {current_best:.4f}")
print(f"\n  Best conditions:")
for feat in SELECTED_FEATURES:
    print(f"    {feat}: {X_current.loc[current_best_idx, feat]:.4f}")


# =============================================================================
# CELL 36: RUN SINGLE BO ITERATION (Template)
# =============================================================================

print("=" * 60)
print("BAYESIAN OPTIMIZATION - ITERATION 1")
print("=" * 60)

iteration = 1

# Step 1: Fit scaler on current data (NO LEAKAGE)
print("\n[Step 1] Fitting scaler on training data...")
scaler_bo = Phase2Scaler(
    continuous_features=CONTINUOUS_FEATURES_SELECTED,
    binary_features=BINARY_FEATURES_SELECTED,
    all_features=SELECTED_FEATURES
)
X_scaled = scaler_bo.fit_transform(X_current)
scaled_bounds = scaler_bo.get_scaled_bounds(bounds_array)

print(f"  ✓ Scaler fitted on {len(X_current)} samples")

# Step 2: Fit Gaussian Process
print("\n[Step 2] Fitting Gaussian Process...")
kernel = (
    ConstantKernel(1.0, constant_value_bounds=(1e-3, 1e3)) * 
    Matern(length_scale=np.ones(len(SELECTED_FEATURES)), 
           length_scale_bounds=(1e-2, 1e2), 
           nu=2.5) + 
    WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5, 1e1))
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=10,
    normalize_y=True,
    random_state=RANDOM_STATE
)

gp.fit(X_scaled.values, y_current.values)

print(f"  ✓ GP fitted")
print(f"  Log-marginal-likelihood: {gp.log_marginal_likelihood_value_:.3f}")

# Step 3: Validate GP with LOO-CV (leakage-free)
print("\n[Step 3] Validating GP (LOO-CV)...")
loo_preds_gp = []
loo_actual_gp = []
loo_stds_gp = []

for train_idx, test_idx in LeaveOneOut().split(X_current):
    # Fit scaler on training fold only
    scaler_temp = Phase2Scaler(
        continuous_features=CONTINUOUS_FEATURES_SELECTED,
        binary_features=BINARY_FEATURES_SELECTED,
        all_features=SELECTED_FEATURES
    )
    X_train_scaled = scaler_temp.fit_transform(X_current.iloc[train_idx])
    X_test_scaled = scaler_temp.transform(X_current.iloc[test_idx])
    
    # Fit GP on training fold
    gp_temp = GaussianProcessRegressor(
        kernel=kernel.clone_with_theta(kernel.theta),
        n_restarts_optimizer=5,
        normalize_y=True,
        random_state=RANDOM_STATE
    )
    gp_temp.fit(X_train_scaled.values, y_current.iloc[train_idx].values)
    
    # Predict on test
    pred, std = gp_temp.predict(X_test_scaled.values, return_std=True)
    loo_preds_gp.append(pred[0])
    loo_stds_gp.append(std[0])
    loo_actual_gp.append(y_current.iloc[test_idx].values[0])

loo_preds_gp = np.array(loo_preds_gp)
loo_stds_gp = np.array(loo_stds_gp)
loo_actual_gp = np.array(loo_actual_gp)

gp_r2 = r2_score(loo_actual_gp, loo_preds_gp)
gp_rmse = np.sqrt(mean_squared_error(loo_actual_gp, loo_preds_gp))

# Calibration
z_score = 1.96
in_ci = np.sum(np.abs(loo_actual_gp - loo_preds_gp) <= z_score * loo_stds_gp)
coverage = in_ci / len(loo_actual_gp)

print(f"  LOO-CV R²: {gp_r2:.4f}")
print(f"  LOO-CV RMSE: {gp_rmse:.4f}")
print(f"  95% CI Coverage: {coverage:.1%}")

# Step 4: Optimize acquisition function
print("\n[Step 4] Optimizing acquisition function...")
next_points_scaled, acq_values = optimize_acquisition_batch(
    gp=gp,
    bounds_scaled=scaled_bounds,
    n_batch=BATCH_SIZE,
    y_best=current_best,
    minimize=not MAXIMIZE_RESPONSE,
    kappa=EXPLORATION_WEIGHT,
    n_restarts=N_OPTIMIZER_RESTARTS,
    min_distance=MIN_DISTANCE_BETWEEN_POINTS
)

print(f"  ✓ Found {len(next_points_scaled)} candidate points")

# Step 5: Convert to original scale
print("\n[Step 5] Converting to original scale...")
next_experiments = []

for i, x_scaled in enumerate(next_points_scaled):
    x_orig = scaler_bo.inverse_transform_point(x_scaled)
    
    row = {'Experiment_ID': f'Iter{iteration}_Exp{i+1}'}
    
    for j, feat in enumerate(SELECTED_FEATURES):
        if feat in BINARY_FEATURES_SELECTED:
            row[feat] = int(round(np.clip(x_orig[j], 0, 1)))
        else:
            bounds_row = bounds_df[bounds_df['feature'] == feat].iloc[0]
            row[feat] = round(np.clip(x_orig[j], bounds_row['min'], bounds_row['max']), 4)
    
    # Add predictions
    mu, sigma = gp.predict(x_scaled.reshape(1, -1), return_std=True)
    row['GP_Predicted_Mean'] = round(mu[0], 4)
    row['GP_Predicted_Std'] = round(sigma[0], 4)
    row['Acquisition_Rank'] = i + 1
    
    next_experiments.append(row)

next_experiments_df = pd.DataFrame(next_experiments)

print("\n" + "=" * 60)
print("PROPOSED EXPERIMENTS FOR ITERATION 1")
print("=" * 60)
print(next_experiments_df.to_string(index=False))


# =============================================================================
# CELL 37: VISUALIZE GP PREDICTIONS
# =============================================================================

print("=" * 60)
print("GP VISUALIZATION")
print("=" * 60)

n_features = len(SELECTED_FEATURES)
fig, axes = plt.subplots(2, n_features, figsize=(4*n_features, 8))

# Row 1: 1D slices of GP
for i, feat in enumerate(SELECTED_FEATURES):
    ax = axes[0, i] if n_features > 1 else axes[0]
    
    if feat in BINARY_FEATURES_SELECTED:
        # Binary: show predictions at 0 and 1
        x_base = X_scaled.mean().values.copy()
        
        preds = []
        stds = []
        for val in [-1, 1]:  # Scaled values
            x_test = x_base.copy()
            x_test[i] = val
            mu, sigma = gp.predict(x_test.reshape(1, -1), return_std=True)
            preds.append(mu[0])
            stds.append(sigma[0])
        
        ax.bar([0, 1], preds, yerr=[1.96*s for s in stds], capsize=5, alpha=0.7, color='steelblue')
        ax.set_xticks([0, 1])
        ax.set_xlabel(f'{feat}')
        
    else:
        # Continuous: show prediction curve
        bounds_row = bounds_df[bounds_df['feature'] == feat].iloc[0]
        x_grid_orig = np.linspace(bounds_row['min'], bounds_row['max'], 50)
        
        x_base = X_scaled.mean().values.copy()
        preds = []
        stds = []
        
        for val in x_grid_orig:
            x_test = x_base.copy()
            x_test[i] = scaler_bo.transform_point(
                [val if j == i else X_current[SELECTED_FEATURES[j]].mean() 
                 for j in range(len(SELECTED_FEATURES))]
            )[i]
            mu, sigma = gp.predict(x_test.reshape(1, -1), return_std=True)
            preds.append(mu[0])
            stds.append(sigma[0])
        
        preds = np.array(preds)
        stds = np.array(stds)
        
        ax.plot(x_grid_orig, preds, 'b-', linewidth=2)
        ax.fill_between(x_grid_orig, preds - 1.96*stds, preds + 1.96*stds, alpha=0.3)
        ax.scatter(X_current[feat], y_current, c='red', s=50, zorder=5, label='Observed')
        
        # Mark proposed points
        for _, row in next_experiments_df.iterrows():
            ax.axvline(row[feat], color='green', linestyle='--', alpha=0.5)
        
        ax.set_xlabel(feat)
    
    ax.set_ylabel(RESPONSE_COLUMN)
    ax.set_title(f'GP Prediction: {feat}')

# Row 2: Validation plots
ax_val1 = axes[1, 0] if n_features > 1 else axes[1]
ax_val1.errorbar(loo_actual_gp, loo_preds_gp, yerr=1.96*loo_stds_gp, 
                  fmt='o', alpha=0.7, capsize=3)
lims = [min(loo_actual_gp.min(), loo_preds_gp.min()), 
        max(loo_actual_gp.max(), loo_preds_gp.max())]
ax_val1.plot(lims, lims, 'r--', linewidth=2)
ax_val1.set_xlabel('Actual')
ax_val1.set_ylabel('Predicted')
ax_val1.set_title(f'LOO-CV: R² = {gp_r2:.3f}')

if n_features > 1:
    ax_val2 = axes[1, 1]
    residuals_gp = loo_actual_gp - loo_preds_gp
    ax_val2.hist(residuals_gp, bins=15, edgecolor='black', alpha=0.7)
    ax_val2.axvline(0, color='red', linestyle='--')
    ax_val2.set_xlabel('Residual')
    ax_val2.set_ylabel('Count')
    ax_val2.set_title('Residual Distribution')

# Hide extra subplots
for j in range(2, n_features):
    axes[1, j].set_visible(False)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / f'phase2_iteration{iteration}_gp.png', dpi=150, bbox_inches='tight')
plt.show()


# =============================================================================
# CELL 38: EXPORT PROPOSED EXPERIMENTS
# =============================================================================

print("=" * 60)
print("EXPORTING PROPOSED EXPERIMENTS")
print("=" * 60)

# Export for lab
export_cols = ['Experiment_ID'] + SELECTED_FEATURES + ['GP_Predicted_Mean', 'GP_Predicted_Std']
export_df = next_experiments_df[export_cols].copy()

# Add column for recording results
export_df[RESPONSE_COLUMN] = ''

export_path = OUTPUT_DIR / f'phase2_iteration{iteration}_experiments.csv'
export_df.to_csv(export_path, index=False)

print(f"✓ Saved: {export_path}")
print(f"\nInstructions:")
print(f"  1. Run these {len(export_df)} experiments in your lab")
print(f"  2. Record the {RESPONSE_COLUMN} values in the last column")
print(f"  3. Save the file")
print(f"  4. Run the next cell to continue optimization")

# Also export as Excel for convenience
export_excel_path = OUTPUT_DIR / f'phase2_iteration{iteration}_experiments.xlsx'
export_df.to_excel(export_excel_path, index=False)
print(f"✓ Also saved as Excel: {export_excel_path}")


# =============================================================================
# CELL 39: UPDATE BO HISTORY
# =============================================================================

print("=" * 60)
print("UPDATING BO HISTORY")
print("=" * 60)

bo_history['iterations'].append(iteration)
bo_history['best_values'].append(current_best)
bo_history['proposed_points'].append(next_experiments_df.to_dict('records'))
bo_history['gp_params'].append({
    'log_marginal_likelihood': gp.log_marginal_likelihood_value_,
    'kernel_params': str(gp.kernel_),
    'loo_r2': gp_r2,
    'loo_rmse': gp_rmse,
    'ci_coverage': coverage,
})

# Save history
history_path = OUTPUT_DIR / 'phase2_bo_history.pkl'
with open(history_path, 'wb') as f:
    pickle.dump(bo_history, f)

history_json_path = OUTPUT_DIR / 'phase2_bo_history.json'
with open(history_json_path, 'w') as f:
    json.dump(bo_history, f, indent=2, default=str)

print(f"✓ Saved: {history_path}")
print(f"✓ Saved: {history_json_path}")

print(f"\nIteration {iteration} Summary:")
print(f"  Current best: {current_best:.4f}")
print(f"  GP LOO-CV R²: {gp_r2:.4f}")
print(f"  Proposed {len(next_experiments_df)} new experiments")

In [ ]:
# =============================================================================
# CELL 40: ITERATION SUMMARY
# =============================================================================

print("=" * 70)
print(f"ITERATION {iteration} COMPLETE")
print("=" * 70)

print(f"""
CURRENT STATE:
  Training samples: {len(X_current)}
  Best observed {RESPONSE_COLUMN}: {current_best:.4f}

GP MODEL QUALITY:
  LOO-CV R²: {gp_r2:.4f}
  LOO-CV RMSE: {gp_rmse:.4f}
  95% CI Coverage: {coverage:.1%}

PROPOSED EXPERIMENTS:
  Count: {len(next_experiments_df)}
  Saved to: {export_path}

NEXT STEPS:
  1. Run the proposed experiments in lab
  2. Record results in the exported CSV/Excel file
  3. Run Cell 41 to input results and continue
""")


# =============================================================================
# CELL 41: INPUT NEW RESULTS AND CONTINUE
# =============================================================================

# ┌─────────────────────────────────────────────────────────────────────────┐
# │ RUN THIS CELL AFTER YOU HAVE COMPLETED THE PROPOSED EXPERIMENTS        │
# │ AND RECORDED THE RESULTS                                                │
# └─────────────────────────────────────────────────────────────────────────┘

print("=" * 60)
print("INPUT NEW EXPERIMENTAL RESULTS")
print("=" * 60)

# Option 1: Load from file (recommended)
results_file = OUTPUT_DIR / f'phase2_iteration{iteration}_experiments.csv'

print(f"\nLooking for results in: {results_file}")

if results_file.exists():
    new_results = pd.read_csv(results_file)
    
    # Check if results are filled in
    if new_results[RESPONSE_COLUMN].isna().all() or (new_results[RESPONSE_COLUMN] == '').all():
        print(f"\n⚠️ No results found in {RESPONSE_COLUMN} column!")
        print(f"   Please fill in the experimental results and re-run this cell.")
        NEW_DATA_READY = False
    else:
        # Convert response to numeric
        new_results[RESPONSE_COLUMN] = pd.to_numeric(new_results[RESPONSE_COLUMN], errors='coerce')
        
        # Check for valid results
        valid_results = new_results[~new_results[RESPONSE_COLUMN].isna()]
        
        if len(valid_results) == 0:
            print(f"\n⚠️ No valid numeric results found!")
            NEW_DATA_READY = False
        else:
            print(f"\n✓ Found {len(valid_results)} valid results")
            print(f"\nNew experimental results:")
            print(valid_results[SELECTED_FEATURES + [RESPONSE_COLUMN]].to_string(index=False))
            NEW_DATA_READY = True
else:
    print(f"\n⚠️ Results file not found: {results_file}")
    print(f"   Please ensure you've saved the results file.")
    NEW_DATA_READY = False

# Option 2: Manual input (alternative)
# Uncomment and modify if you prefer manual entry
# new_results = pd.DataFrame({
#     'Feature_A': [value1, value2, ...],
#     'Feature_B': [value1, value2, ...],
#     RESPONSE_COLUMN: [result1, result2, ...],
# })
# NEW_DATA_READY = True


# =============================================================================
# CELL 42: UPDATE DATASET WITH NEW RESULTS
# =============================================================================

# ┌─────────────────────────────────────────────────────────────────────────┐
# │ RUN THIS CELL ONLY IF NEW_DATA_READY = True                            │
# └─────────────────────────────────────────────────────────────────────────┘

if 'NEW_DATA_READY' in dir() and NEW_DATA_READY:
    print("=" * 60)
    print("UPDATING DATASET")
    print("=" * 60)
    
    # Extract new data points
    X_new = valid_results[SELECTED_FEATURES].copy()
    y_new = valid_results[RESPONSE_COLUMN].copy()
    
    # Append to current dataset
    X_current = pd.concat([X_current, X_new], ignore_index=True)
    y_current = pd.concat([y_current, y_new], ignore_index=True)
    
    # Update best
    if MAXIMIZE_RESPONSE:
        new_best = y_current.max()
        new_best_idx = y_current.idxmax()
    else:
        new_best = y_current.min()
        new_best_idx = y_current.idxmin()
    
    improvement = current_best - new_best if not MAXIMIZE_RESPONSE else new_best - current_best
    
    print(f"\nDataset updated:")
    print(f"  Previous samples: {len(X_current) - len(X_new)}")
    print(f"  New samples: {len(X_new)}")
    print(f"  Total samples: {len(X_current)}")
    
    print(f"\nBest value update:")
    print(f"  Previous best: {current_best:.4f}")
    print(f"  New best: {new_best:.4f}")
    print(f"  Improvement: {improvement:.4f}")
    
    if improvement > 0:
        print(f"\n✓ IMPROVEMENT FOUND!")
        print(f"  New best conditions:")
        for feat in SELECTED_FEATURES:
            print(f"    {feat}: {X_current.loc[new_best_idx, feat]:.4f}")
    else:
        print(f"\n⚠️ No improvement in this iteration")
    
    # Update tracking
    bo_history['observed_responses'].append(y_new.tolist())
    
    # Update current best
    previous_best = current_best
    current_best = new_best
    current_best_idx = new_best_idx
    
    # Save updated data
    updated_data = X_current.copy()
    updated_data[RESPONSE_COLUMN] = y_current.values
    updated_data.to_csv(OUTPUT_DIR / 'phase2_all_data.csv', index=False)
    print(f"\n✓ Saved updated dataset: {OUTPUT_DIR / 'phase2_all_data.csv'}")
    
    # Increment iteration
    iteration += 1
    
    print(f"\n{'='*60}")
    print(f"READY FOR ITERATION {iteration}")
    print(f"{'='*60}")
    print(f"Run Cell 43 to continue optimization or Cell 44 to check convergence")
    
else:
    print("\n⚠️ NEW_DATA_READY is False. Please complete Cell 41 first.")


# =============================================================================
# CELL 43: RUN NEXT BO ITERATION
# =============================================================================

# ┌─────────────────────────────────────────────────────────────────────────┐
# │ RUN THIS CELL TO PERFORM ANOTHER BO ITERATION                          │
# │ This is essentially a repeat of Cells 36-40 with updated data          │
# └─────────────────────────────────────────────────────────────────────────┘

print("=" * 70)
print(f"BAYESIAN OPTIMIZATION - ITERATION {iteration}")
print("=" * 70)

# Step 1: Fit scaler on current data (NO LEAKAGE)
print("\n[Step 1] Fitting scaler on training data...")
scaler_bo = Phase2Scaler(
    continuous_features=CONTINUOUS_FEATURES_SELECTED,
    binary_features=BINARY_FEATURES_SELECTED,
    all_features=SELECTED_FEATURES
)
X_scaled = scaler_bo.fit_transform(X_current)
scaled_bounds = scaler_bo.get_scaled_bounds(bounds_array)
print(f"  ✓ Scaler fitted on {len(X_current)} samples")

# Step 2: Fit Gaussian Process
print("\n[Step 2] Fitting Gaussian Process...")
kernel = (
    ConstantKernel(1.0, constant_value_bounds=(1e-3, 1e3)) * 
    Matern(length_scale=np.ones(len(SELECTED_FEATURES)), 
           length_scale_bounds=(1e-2, 1e2), 
           nu=2.5) + 
    WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5, 1e1))
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=10,
    normalize_y=True,
    random_state=RANDOM_STATE
)

gp.fit(X_scaled.values, y_current.values)
print(f"  ✓ GP fitted")
print(f"  Log-marginal-likelihood: {gp.log_marginal_likelihood_value_:.3f}")

# Step 3: Validate GP (leakage-free LOO-CV)
print("\n[Step 3] Validating GP (LOO-CV)...")
loo_preds_gp = []
loo_actual_gp = []
loo_stds_gp = []

for train_idx, test_idx in LeaveOneOut().split(X_current):
    scaler_temp = Phase2Scaler(
        continuous_features=CONTINUOUS_FEATURES_SELECTED,
        binary_features=BINARY_FEATURES_SELECTED,
        all_features=SELECTED_FEATURES
    )
    X_train_scaled = scaler_temp.fit_transform(X_current.iloc[train_idx])
    X_test_scaled = scaler_temp.transform(X_current.iloc[test_idx])
    
    gp_temp = GaussianProcessRegressor(
        kernel=kernel.clone_with_theta(kernel.theta),
        n_restarts_optimizer=5,
        normalize_y=True,
        random_state=RANDOM_STATE
    )
    gp_temp.fit(X_train_scaled.values, y_current.iloc[train_idx].values)
    
    pred, std = gp_temp.predict(X_test_scaled.values, return_std=True)
    loo_preds_gp.append(pred[0])
    loo_stds_gp.append(std[0])
    loo_actual_gp.append(y_current.iloc[test_idx].values[0])

loo_preds_gp = np.array(loo_preds_gp)
loo_stds_gp = np.array(loo_stds_gp)
loo_actual_gp = np.array(loo_actual_gp)

gp_r2 = r2_score(loo_actual_gp, loo_preds_gp)
gp_rmse = np.sqrt(mean_squared_error(loo_actual_gp, loo_preds_gp))
in_ci = np.sum(np.abs(loo_actual_gp - loo_preds_gp) <= 1.96 * loo_stds_gp)
coverage = in_ci / len(loo_actual_gp)

print(f"  LOO-CV R²: {gp_r2:.4f}")
print(f"  LOO-CV RMSE: {gp_rmse:.4f}")
print(f"  95% CI Coverage: {coverage:.1%}")

# Step 4: Optimize acquisition function
print("\n[Step 4] Optimizing acquisition function...")
next_points_scaled, acq_values = optimize_acquisition_batch(
    gp=gp,
    bounds_scaled=scaled_bounds,
    n_batch=BATCH_SIZE,
    y_best=current_best,
    minimize=not MAXIMIZE_RESPONSE,
    kappa=EXPLORATION_WEIGHT,
    n_restarts=N_OPTIMIZER_RESTARTS,
    min_distance=MIN_DISTANCE_BETWEEN_POINTS
)
print(f"  ✓ Found {len(next_points_scaled)} candidate points")

# Step 5: Convert to original scale
print("\n[Step 5] Converting to original scale...")
next_experiments = []

for i, x_scaled in enumerate(next_points_scaled):
    x_orig = scaler_bo.inverse_transform_point(x_scaled)
    
    row = {'Experiment_ID': f'Iter{iteration}_Exp{i+1}'}
    
    for j, feat in enumerate(SELECTED_FEATURES):
        if feat in BINARY_FEATURES_SELECTED:
            row[feat] = int(round(np.clip(x_orig[j], 0, 1)))
        else:
            bounds_row = bounds_df[bounds_df['feature'] == feat].iloc[0]
            row[feat] = round(np.clip(x_orig[j], bounds_row['min'], bounds_row['max']), 4)
    
    mu, sigma = gp.predict(x_scaled.reshape(1, -1), return_std=True)
    row['GP_Predicted_Mean'] = round(mu[0], 4)
    row['GP_Predicted_Std'] = round(sigma[0], 4)
    row['Acquisition_Rank'] = i + 1
    
    next_experiments.append(row)

next_experiments_df = pd.DataFrame(next_experiments)

print("\n" + "=" * 60)
print(f"PROPOSED EXPERIMENTS FOR ITERATION {iteration}")
print("=" * 60)
print(next_experiments_df.to_string(index=False))

# Export
export_cols = ['Experiment_ID'] + SELECTED_FEATURES + ['GP_Predicted_Mean', 'GP_Predicted_Std']
export_df = next_experiments_df[export_cols].copy()
export_df[RESPONSE_COLUMN] = ''

export_path = OUTPUT_DIR / f'phase2_iteration{iteration}_experiments.csv'
export_df.to_csv(export_path, index=False)
export_df.to_excel(OUTPUT_DIR / f'phase2_iteration{iteration}_experiments.xlsx', index=False)

print(f"\n✓ Saved: {export_path}")

# Update history
bo_history['iterations'].append(iteration)
bo_history['best_values'].append(current_best)
bo_history['proposed_points'].append(next_experiments_df.to_dict('records'))
bo_history['gp_params'].append({
    'log_marginal_likelihood': gp.log_marginal_likelihood_value_,
    'kernel_params': str(gp.kernel_),
    'loo_r2': gp_r2,
    'loo_rmse': gp_rmse,
    'ci_coverage': coverage,
})

# Save history
with open(OUTPUT_DIR / 'phase2_bo_history.pkl', 'wb') as f:
    pickle.dump(bo_history, f)
with open(OUTPUT_DIR / 'phase2_bo_history.json', 'w') as f:
    json.dump(bo_history, f, indent=2, default=str)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

ax1 = axes[0]
ax1.errorbar(loo_actual_gp, loo_preds_gp, yerr=1.96*loo_stds_gp, fmt='o', alpha=0.7, capsize=3)
lims = [min(loo_actual_gp.min(), loo_preds_gp.min()), max(loo_actual_gp.max(), loo_preds_gp.max())]
ax1.plot(lims, lims, 'r--', linewidth=2)
ax1.set_xlabel('Actual')
ax1.set_ylabel('Predicted')
ax1.set_title(f'LOO-CV: R² = {gp_r2:.3f}')

ax2 = axes[1]
ax2.plot(bo_history['iterations'], bo_history['best_values'], 'bo-', linewidth=2, markersize=8)
ax2.set_xlabel('Iteration')
ax2.set_ylabel(f'Best {RESPONSE_COLUMN}')
ax2.set_title('Optimization Progress')
ax2.grid(True, alpha=0.3)

ax3 = axes[2]
r2_values = [p['loo_r2'] for p in bo_history['gp_params']]
ax3.plot(bo_history['iterations'], r2_values, 'go-', linewidth=2, markersize=8)
ax3.set_xlabel('Iteration')
ax3.set_ylabel('LOO-CV R²')
ax3.set_title('GP Model Quality')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / f'phase2_iteration{iteration}_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n{'='*60}")
print(f"ITERATION {iteration} COMPLETE")
print(f"{'='*60}")
print(f"Current best: {current_best:.4f}")
print(f"Run experiments, then go back to Cell 41 to input results")


# =============================================================================
# CELL 44: CHECK CONVERGENCE
# =============================================================================

print("=" * 70)
print("CONVERGENCE CHECK")
print("=" * 70)

if len(bo_history['best_values']) >= 2:
    best_values = bo_history['best_values']
    iterations = bo_history['iterations']
    
    # Calculate improvements
    improvements = []
    for i in range(1, len(best_values)):
        if MAXIMIZE_RESPONSE:
            imp = best_values[i] - best_values[i-1]
        else:
            imp = best_values[i-1] - best_values[i]
        improvements.append(imp)
    
    print(f"\nOptimization History:")
    print("-" * 50)
    for i, (it, bv) in enumerate(zip(iterations, best_values)):
        imp_str = f" (Δ = {improvements[i-1]:+.4f})" if i > 0 else ""
        print(f"  Iteration {it}: Best = {bv:.4f}{imp_str}")
    
    # Check convergence criteria
    print(f"\nConvergence Analysis:")
    print("-" * 50)
    
    # Criterion 1: Recent improvement
    recent_improvements = improvements[-PATIENCE:] if len(improvements) >= PATIENCE else improvements
    max_recent_improvement = max(recent_improvements) if recent_improvements else 0
    
    converged_improvement = max_recent_improvement < IMPROVEMENT_THRESHOLD
    print(f"  Max improvement in last {PATIENCE} iterations: {max_recent_improvement:.4f}")
    print(f"  Threshold: {IMPROVEMENT_THRESHOLD}")
    print(f"  Converged by improvement? {'YES ✓' if converged_improvement else 'NO'}")
    
    # Criterion 2: Relative improvement
    total_range = max(best_values) - min(best_values) if len(best_values) > 1 else 0
    relative_improvement = max_recent_improvement / (total_range + 1e-8)
    
    converged_relative = relative_improvement < 0.01
    print(f"\n  Relative improvement: {relative_improvement:.2%}")
    print(f"  Converged by relative improvement? {'YES ✓' if converged_relative else 'NO'}")
    
    # Criterion 3: GP uncertainty at best point
    if 'gp' in dir():
        x_best = X_current.iloc[current_best_idx].values
        x_best_scaled = scaler_bo.transform_point(x_best)
        _, sigma_best = gp.predict(x_best_scaled.reshape(1, -1), return_std=True)
        
        relative_uncertainty = sigma_best[0] / (y_current.std() + 1e-8)
        converged_uncertainty = relative_uncertainty < 0.1
        
        print(f"\n  GP uncertainty at best point: {sigma_best[0]:.4f}")
        print(f"  Relative to response std: {relative_uncertainty:.2%}")
        print(f"  Converged by uncertainty? {'YES ✓' if converged_uncertainty else 'NO'}")
    else:
        converged_uncertainty = False
    
    # Overall convergence
    n_criteria_met = sum([converged_improvement, converged_relative, converged_uncertainty])
    CONVERGED = n_criteria_met >= 2
    
    print(f"\n{'='*50}")
    print(f"CONVERGENCE STATUS: {'CONVERGED ✓' if CONVERGED else 'NOT CONVERGED'}")
    print(f"{'='*50}")
    print(f"  Criteria met: {n_criteria_met}/3")
    
    if CONVERGED:
        print(f"\n  Recommendation: Proceed to confirmation runs (Cell 45)")
    else:
        print(f"\n  Recommendation: Continue optimization (Cell 43)")
        print(f"  Iterations completed: {len(iterations)}")
        print(f"  Max iterations: {MAX_ITERATIONS}")
        
        if len(iterations) >= MAX_ITERATIONS:
            print(f"\n  ⚠️ Maximum iterations reached!")
            print(f"     Consider increasing MAX_ITERATIONS or accepting current best")
else:
    print("Not enough iterations to check convergence.")
    print("Complete at least 2 iterations first.")
    CONVERGED = False


# =============================================================================
# CELL 45: FINAL RESULTS AND CONFIRMATION RUNS
# =============================================================================

print("=" * 70)
print("FINAL RESULTS")
print("=" * 70)

print(f"""
OPTIMIZATION COMPLETE

BEST RESULT FOUND:
  {RESPONSE_COLUMN}: {current_best:.4f}
  
OPTIMAL CONDITIONS:
""")

for feat in SELECTED_FEATURES:
    value = X_current.loc[current_best_idx, feat]
    bounds_row = bounds_df[bounds_df['feature'] == feat].iloc[0]
    
    if feat in BINARY_FEATURES_SELECTED:
        # Decode binary
        if feat in BINARY_MAPPINGS:
            mapping = BINARY_MAPPINGS[feat]
            original_value = [k for k, v in mapping.items() if v == value][0]
            print(f"  {feat}: {int(value)} (original: {original_value})")
        else:
            print(f"  {feat}: {int(value)}")
    else:
        print(f"  {feat}: {value:.4f}")
        print(f"      Bounds: [{bounds_row['min']:.4f}, {bounds_row['max']:.4f}]")

# GP prediction at optimum
if 'gp' in dir() and 'scaler_bo' in dir():
    x_opt = X_current.loc[current_best_idx, SELECTED_FEATURES].values
    x_opt_scaled = scaler_bo.transform_point(x_opt)
    mu_opt, sigma_opt = gp.predict(x_opt_scaled.reshape(1, -1), return_std=True)
    
    print(f"\nGP PREDICTION AT OPTIMUM:")
    print(f"  Predicted mean: {mu_opt[0]:.4f}")
    print(f"  Predicted std: {sigma_opt[0]:.4f}")
    print(f"  95% CI: [{mu_opt[0] - 1.96*sigma_opt[0]:.4f}, {mu_opt[0] + 1.96*sigma_opt[0]:.4f}]")

# Generate confirmation runs
print(f"\n{'='*60}")
print("CONFIRMATION RUNS")
print("=" * 60)

n_confirmation = 5
confirmation_runs = []

for i in range(n_confirmation):
    run = {'Run_ID': f'Confirm_{i+1}'}
    for feat in SELECTED_FEATURES:
        run[feat] = X_current.loc[current_best_idx, feat]
    run[f'Expected_{RESPONSE_COLUMN}'] = current_best
    confirmation_runs.append(run)

confirmation_df = pd.DataFrame(confirmation_runs)
confirmation_df[f'Actual_{RESPONSE_COLUMN}'] = ''

print(f"\nRun {n_confirmation} confirmation experiments at optimal conditions:")
print(confirmation_df.to_string(index=False))

# Save confirmation runs
confirmation_path = OUTPUT_DIR / 'phase2_confirmation_runs.csv'
confirmation_df.to_csv(confirmation_path, index=False)
confirmation_df.to_excel(OUTPUT_DIR / 'phase2_confirmation_runs.xlsx', index=False)

print(f"\n✓ Saved: {confirmation_path}")


# =============================================================================
# CELL 46: OPTIMIZATION SUMMARY REPORT
# =============================================================================

print("=" * 70)
print("OPTIMIZATION SUMMARY REPORT")
print("=" * 70)

# Plot optimization history
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Best value over iterations
ax1 = axes[0, 0]
ax1.plot(bo_history['iterations'], bo_history['best_values'], 'bo-', linewidth=2, markersize=10)
ax1.set_xlabel('Iteration')
ax1.set_ylabel(f'Best {RESPONSE_COLUMN}')
ax1.set_title('Optimization Progress')
ax1.grid(True, alpha=0.3)

# Add improvement annotations
for i in range(1, len(bo_history['best_values'])):
    if MAXIMIZE_RESPONSE:
        imp = bo_history['best_values'][i] - bo_history['best_values'][i-1]
    else:
        imp = bo_history['best_values'][i-1] - bo_history['best_values'][i]
    if imp > 0:
        ax1.annotate(f'+{imp:.3f}', 
                     xy=(bo_history['iterations'][i], bo_history['best_values'][i]),
                     xytext=(5, 10), textcoords='offset points', fontsize=8, color='green')

# Plot 2: GP R² over iterations
ax2 = axes[0, 1]
r2_values = [p['loo_r2'] for p in bo_history['gp_params']]
ax2.plot(bo_history['iterations'], r2_values, 'go-', linewidth=2, markersize=10)
ax2.set_xlabel('Iteration')
ax2.set_ylabel('LOO-CV R²')
ax2.set_title('GP Model Quality Over Time')
ax2.set_ylim([0, 1])
ax2.grid(True, alpha=0.3)

# Plot 3: All observed data
ax3 = axes[1, 0]
ax3.hist(y_current, bins=20, edgecolor='black', alpha=0.7, color='steelblue')
ax3.axvline(current_best, color='red', linewidth=2, linestyle='--', label=f'Best: {current_best:.4f}')
ax3.set_xlabel(RESPONSE_COLUMN)
ax3.set_ylabel('Frequency')
ax3.set_title(f'Distribution of All Observed {RESPONSE_COLUMN}')
ax3.legend()

# Plot 4: Feature importances (from correlation)
ax4 = axes[1, 1]
correlations_final = X_current.corrwith(y_current)
colors = ['forestgreen' if c > 0 else 'crimson' for c in correlations_final]
ax4.barh(SELECTED_FEATURES, np.abs(correlations_final), color=colors)
ax4.set_xlabel('|Correlation|')
ax4.set_title('Feature-Response Correlations (Final Dataset)')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'phase2_final_summary.png', dpi=150, bbox_inches='tight')
plt.show()

# Print summary statistics
print(f"""
SUMMARY STATISTICS:
───────────────────────────────────────────────────────────────

EXPERIMENT COUNT:
  Phase 1 (feature selection): {phase1_checkpoint['metadata']['n_initial_experiments']}
  Phase 2 (optimization): {len(X_current)}
  Total experiments: {phase1_checkpoint['metadata']['n_initial_experiments'] + len(X_current)}

OPTIMIZATION:
  Iterations completed: {len(bo_history['iterations'])}
  Starting best: {bo_history['best_values'][0]:.4f}
  Final best: {current_best:.4f}
  Total improvement: {abs(bo_history['best_values'][0] - current_best):.4f}

GP MODEL (Final):
  LOO-CV R²: {bo_history['gp_params'][-1]['loo_r2']:.4f}
  LOO-CV RMSE: {bo_history['gp_params'][-1]['loo_rmse']:.4f}
  95% CI Coverage: {bo_history['gp_params'][-1]['ci_coverage']:.1%}

OPTIMAL CONDITIONS:
""")

for feat in SELECTED_FEATURES:
    value = X_current.loc[current_best_idx, feat]
    print(f"  {feat}: {value:.4f}" if feat not in BINARY_FEATURES_SELECTED else f"  {feat}: {int(value)}")

print(f"""
FILES GENERATED:
  • {OUTPUT_DIR}/phase1_checkpoint.json - Phase 1 settings
  • {OUTPUT_DIR}/phase1_feature_info.csv - Feature details
  • {OUTPUT_DIR}/phase2_all_data.csv - All experimental data
  • {OUTPUT_DIR}/phase2_bo_history.json - Optimization history
  • {OUTPUT_DIR}/phase2_confirmation_runs.csv - Confirmation experiments
  • Various PNG plots
""")


# =============================================================================
# CELL 47: EXPORT FINAL REPORT
# =============================================================================

print("=" * 70)
print("EXPORTING FINAL REPORT")
print("=" * 70)

# Create comprehensive final report
final_report = {
    'optimization_summary': {
        'objective': 'Maximize' if MAXIMIZE_RESPONSE else 'Minimize',
        'response_variable': RESPONSE_COLUMN,
        'final_best_value': float(current_best),
        'total_iterations': len(bo_history['iterations']),
        'total_experiments_phase2': len(X_current),
    },
    'optimal_conditions': {
        feat: float(X_current.loc[current_best_idx, feat]) if feat not in BINARY_FEATURES_SELECTED 
              else int(X_current.loc[current_best_idx, feat])
        for feat in SELECTED_FEATURES
    },
    'feature_selection': {
        'method': 'Consensus of Correlation, Lasso, and PLS-VIP',
        'selected_features': SELECTED_FEATURES,
        'binary_features': BINARY_FEATURES_SELECTED,
        'continuous_features': CONTINUOUS_FEATURES_SELECTED,
    },
    'optimization_history': {
        'iterations': bo_history['iterations'],
        'best_values': bo_history['best_values'],
    },
    'final_gp_model': bo_history['gp_params'][-1] if bo_history['gp_params'] else {},
    'bounds': bounds_df.to_dict('records'),
    'binary_mappings': {k: {str(k2): v2 for k2, v2 in v.items()} 
                        for k, v in BINARY_MAPPINGS.items() if k in SELECTED_FEATURES},
}

# Save final report
report_path = OUTPUT_DIR / 'final_optimization_report.json'
with open(report_path, 'w') as f:
    json.dump(final_report, f, indent=2, default=str)

print(f"✓ Saved: {report_path}")

# Save all data
all_data_path = OUTPUT_DIR / 'all_experimental_data.csv'
final_data = X_current.copy()
final_data[RESPONSE_COLUMN] = y_current.values
final_data['Source'] = ['Phase2'] * len(final_data)
final_data.to_csv(all_data_path, index=False)

print(f"✓ Saved: {all_data_path}")

print(f"""
{'='*70}
BAYESIAN OPTIMIZATION PIPELINE COMPLETE
{'='*70}

Next Steps:
1. Run confirmation experiments from: phase2_confirmation_runs.csv
2. Compare confirmation results to predicted optimal
3. If confirmation successful → implement optimal conditions
4. If confirmation fails → review and potentially continue optimization

Thank you for using the Integrated BO Pipeline!
{'='*70}
""")